In [0]:
%run ./Includes

In [0]:
connectionString = "Endpoint=sb://cust-dev-eventhub-ns.servicebus.windows.net/;SharedAccessKeyName=sender;SharedAccessKey=Kappq9Y/s+FjwEePbtMnINsv7fKHhjocn+AEhPD0yIc=;EntityPath=payment"

ehConf = {
    'eventhubs.connectionString': sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString)
}

In [0]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import *

df = spark.readStream \
    .format("eventhubs") \
    .options(**ehConf) \
    .load()

payments = df.selectExpr("cast(body as string) as json_data")
display(payments)

In [0]:
load_bronze_multiplex(payments,"payments","/mnt/checkpoint/payments/","/mnt/bronze/multiplexbronzetable")

In [0]:
%sql
select * from delta.`/mnt/bronze/multiplexbronzetable` where source='payments';